# Draft for HW 5

In [1]:
import cv2 
import math
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from scipy import linalg


Note that the data for the Yale faces datasets can be found on the course website in the "Homework + Data + Text" section: https://faculty.washington.edu/kutz/am584/am584.html. 
        

In [2]:
main_dir = '/Users/jmnugent/Documents/__Year_3_2020-2021/AMATH_584-Numerical_Linear_Algebra/Homework/python/'

crop_dir = main_dir + 'CroppedYale/'
uncrop_dir = main_dir + 'yalefaces_uncropped/yalefaces/'
save_dir = main_dir + 'amath584/hw5_iterative_methods/'


## Define Functions

In [3]:
def power_iteration(A, tol=1e-6, num_iter=None, stop=1e6):
    """
    Performs power iteration on input matrix A (m x m) until the
    gradient of the Rayleigh quotient is less than or equal to the
    tol (default: 1e-6). If num_iter is given, will perform a set 
    number of iterations (and not necessarily go until the tolerance is reached).
    
    Assumes A is squre. The initial guess is a randomized vector of unit length.
    
    Returns the largest eigenvalue and its corresponding eigenvector after iteration is
    complete. If num_iter=None, also returns how many iterations it took to
    reach the tolerance.
    
    Stops after 'stop' iterations to prevent an infinite loop.
    """
    # functions for the Rayleigh quotient and its gradient
    r = lambda x: (x.T @ A @ x) / (x.T @ x)
    gradr = lambda x: (2 / (x.T @ x)) * ((A @ x) - (r(x)*x))
    
    # get size m
    m = A.shape[0]

    # iterate some number of times
    if num_iter is not None:
        if type(num_iter) != int:
            raise Exception('Invalid input for num_iter! Must be an integer # of iterations.')
        else:
            # initialize
            lk = np.zeros(num_iter)
            vk = np.zeros((num_iter, m, 1))
            
            v0 = np.random.randn(m, 1)
            v0 = v0 / np.linalg.norm(v0)
            vk[0, :] = v0
            lk[0] = v0.T @ A @ v0
            
            # iterate
            for k in range(num_iter-1):
                w = A @ vk[k, :]
                vk[k+1, :] = w / np.linalg.norm(w)
                lk[k+1] = vk[k+1, :].T @ A @ vk[k+1, :]

            return [lk[-1], vk[-1, :]]
    
    # iterate until you reach some value
    else: 
        # initialize
        n = 0
        vk = np.random.randn(m, 1)
        vk = vk / np.linalg.norm(vk)
        lk = vk.T @ A @ vk
        
        # iterate
        while np.any(np.abs(gradr(vk)) > tol):
            if n > stop:
                print('STOPPED AFTER ' + str(int(stop)) + ' ITERATIONS')
                return [float(lk), vk, n]
                break
            n += 1
            w = A @ vk
            vk = w / np.linalg.norm(w)
            lk = vk.T @ A @ vk
        
        return [float(lk), vk, n]
    
    

In [4]:
def rq_iteration(A, v0, tol=1e-6, num_iter=None, stop=1e6):
    """
    Performs Rayleigh Quotient iteration on input matrix A (m x m)
    for some initial guess of eigenvector v0 (m x 1) until the difference
    between lambda(k-1) and lambda(k) < tol. If num_iter is given, will
    perform a set number of iterations (and not necessarily go until the
    tolerance is reached).
    
    Assumes A is square.
    
    Returns the found eigenvalue and its corresponding eigenvector after iteration is
    complete. If num_iter=None, also returns how many iterations it took to
    reach the tolerance.
    
    Stops after 'stop' iterations to prevent an infinite loop.
    """
    # functions for the Rayleigh quotient and its gradient
    r = lambda x: (x.T @ A @ x) / (x.T @ x)
    gradr = lambda x: (2 / (x.T @ x)) * ((A @ x) - (r(x)*x))
    
    # get size m
    m = A.shape[0]
    
    # initialize arrays
    eigs = np.zeros(m)
    vecs = np.zeros((m, m))
    
    # iterate some number of times
    if num_iter is not None:
        if type(num_iter) != int:
            raise Exception('Invalid input for num_iter! Must be an integer # of iterations.')
        else:
            # initialize
            lk = np.zeros(num_iter)
            vk = np.zeros((num_iter, m, 1))
            vk[0, :] = v0
            lk[0] = v0.T @ A @ v0
            
            # iterate
            for k in range(num_iter-1):
                w = np.linalg.solve((A - lk[k]*np.eye(m)), vk[k, :])
                vk[k+1, :] = w / np.linalg.norm(w)
                lk[k+1] = vk[k+1, :].T @ A @ vk[k+1, :]
                
            return [lk[-1], vk[-1, :]]
            
    # iterate until the difference is very small
    else:
        # initialize
        l0 = v0.T @ A @ v0
        
        # first iteration
        n = 1
        w = np.linalg.solve((A - l0*np.eye(m)), v0)
        vk = w / np.linalg.norm(w)
        lk = vk.T @ A @ vk
        
        # iterate until lambdas are equal within the tolerance
        lk_prev = l0
        while np.abs(lk - lk_prev) > tol:
            n += 1
            if n > stop:
                print('STOPPED AFTER ' + str(int(stop)) + ' ITERATIONS')
                return [lk[0][0], vk, n]
                break
            lk_prev = lk
            try:
                w = np.linalg.solve((A - lk*np.eye(m)), vk)
            except:
                return [float(lk), vk, n]
                break
            vk = w / np.linalg.norm(w)
            lk = vk.T @ A @ vk            

        return [lk[0][0], vk, n]
    

In [5]:
# function to find gradient of the rayleigh quotient
r = lambda x: (x.T @ A @ x) / (x.T @ x)
gradr = lambda x: (2 / (x.T @ x)) * ((A @ x) - (r(x)*x))
   
# to check for accuracy, this should be true
tol = 1e-6
is_close = lambda x: np.all(np.abs(gradr(x)) < tol)


# 1. Eigenvalues and Power Iterations

## part (a)
Generate a random, symmetric matrix A which is mxm where m=10. Use the EIGS command (**scipy.linalg.eig**) to give you the _ground truth_ eigenvalues and eigenvectors

Used suggestion from here to make the random matrix symmetric: https://stackoverflow.com/questions/10806790/generating-symmetric-matrices-in-numpy

In [ ]:
# build the matrix
m = 10
A_nonsymm = np.random.randn(m, m)
A = np.tril(A_nonsymm) + np.tril(A_nonsymm, -1).T

# check symmetric
print(np.all(A == A.T))

# ground truth
eigvals, eigvecs = scipy.linalg.eig(A)


In [ ]:
print(eigvals)

## part (b)
Find the largest eigenvalue with the **power iteration** method. Compare the accuracy of the method as a function of iterations.

##### Test it on a few things!

In [ ]:
lambda1, v1, n = power_iteration(A, tol=tol)
print(lambda1)
print(is_close(v1))
print(n, 'iterations')

lambda2, v2 = power_iteration(A, num_iter=100)
print('\n', lambda2)
print(is_close(v2))
print('100 iterations')

lambda3, v3 = power_iteration(A, num_iter=10000)
print('\n', lambda3)
print(is_close(v3))
print('10,000 iterations')


In [ ]:
# the true largest eigenvalue
print(np.max(np.abs(eigvals)))

### Make plots

"Find the largest eigenvalue with the **power iteration** method. Compare the accuracy of the method as a function of iterations." (_What happens to v(k) as k gets big? What is v converging to?_)

In [ ]:
# TODO

## part (c)

Find all 10 eigenvalues by **Rayleigh Quotient** iteration and guessing initial "eigenvectors." Compare the accuracy of the method as a function of iterations and discuss your initial guesses to find all eigenvalue/eigenvector pairs.

##### ...actually guess...!!

In [ ]:
# ?????

##### Loop to get all 10 unique values
NOTE: this only works as written for symmetric matrices A because you know the eigenvalues are unique!

In [ ]:
def rand_unitvec(m):
    """Generate a random mx1 vector of unit length.
    """
    v = np.random.randn(m, 1)
    v1 = v / np.linalg.norm(v)
    
    return v1


In [ ]:
# initialize list of eigenvalues with an array because it
# gets returned as a float
eigs_rq = np.zeros(1)

# perform one iteration to find an eigenvalue/eigenvector pair
eigs_rq[0], vecs_rq, n = rq_iteration(A, rand_unitvec(m))
print(n, 'iterations')

# repeat the iteration until you find all 10 unique ones;
# check for uniqueness to 6 decimal points (because tol=1e-6)
while len(eigs_rq) < 10:
    eig, vecs, nn = rq_iteration(A, rand_unitvec(m), tol=1e-6)
    if np.round(eig, 6) not in np.round(eigs_rq, 6):
        print(n, 'iterations')
        eigs_rq = np.append(eigs_rq, eig)
        vecs_rq = np.append(vecs_rq, vecs, axis=-1)
        

In [ ]:
# try with some others...
e2, v2 = rq_iteration(A, rand_unitvec(m), num_iter=2)
print(e2)

e3, v3 = rq_iteration(A, rand_unitvec(m), num_iter=3)
print(e3)

e5, v5 = rq_iteration(A, rand_unitvec(m), num_iter=5)
print(e5)


##### quick check that you got all 10!

In [ ]:
print('From Rayleigh Quotient iteration:')
# print(sorted(eigs_rq, key=abs)[::-1])
for x in eigs_rq: print(x)

print('\nGround truth:')
for x in eigvals: print(x)


### Plot...

In [ ]:
# TODO

## part (d)
Repeat (b) and (c) with a **random matrix** that is not symmetric. Be sure to plot the eigenvalue in the complex plane.

In [ ]:
# build the matrix
m = 10
A_ns = np.random.randn(m, m)

# ground truth
eigvals_ns, eigvecs_ns = scipy.linalg.eig(A_ns)


In [ ]:
print(eigvals_ns)

#### part (b), non-symmetric

In [ ]:
# I think I need to do some kind of orthogonalization first??
# check notes in AM

In [ ]:
lambda1_ns, v1_ns = power_iteration(A_ns, tol=tol, num_iter=10000)
print(lambda1_ns)
print(is_close(v1_ns))
print(gradr(v1_ns))


In [ ]:
%%time 

lambda2_ns, v2_ns, n2_ns = power_iteration(A_ns, tol=1e-6, stop=1e6)
print(lambda2_ns)
print(is_close(v2_ns))
print(gradr(v2_ns))


In [ ]:
# ...not sure what the difference here will be... just more iterations
# needed? 

# do I need to make sure the initial guess is complex??


#### part (c), non-symmetric

In [ ]:
%%time
# ACTUALLY GUESS for Rayleigh stuff! (not this)

v0c = np.random.randn(m, 1) + np.random.randn(m, 1)*1j
v0c = v0c / np.linalg.norm(v0c)

te, tv, tn = rq_iteration(A_ns, v0c, tol=1e-6, stop=1e6)
print(te)
print(tn)


In [ ]:
# ALSO NOT SURE WHAT TO DO HERE!
# ...because how can you get the eigenvecs/eigenvals? Just stop after
# some limit??

# 2. Yale Faces

## Read in the data:

### Cropped:

In [12]:
# get a list of paths to each subfolder in CroppedYale
paths = [crop_dir + dirname for dirname in os.listdir(crop_dir)
         if os.path.isdir(os.path.join(crop_dir, dirname))]

# initialize list to hold the averaged data matrices for each image
n_img = len(paths)
cropped_pics = [[]]*n_img
cropped_avgs = [[]]*n_img

for i in range(n_img):
    # get the list of file names within the subfolder for that image
    subfolder = paths[i] + '/'
    imagenames = [subfolder + f for f in os.listdir(subfolder)
                  if os.path.isfile(os.path.join(subfolder, f))]
    
    # make one list containing the data matrices for each (grayscale) image 
    cropped_pics[i] = [cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2GRAY) for x in imagenames]

    # averaged the data matrix for this image and add to the list
    cropped_avgs[i] = np.mean(cropped_pics[i], axis=0)
    

In [7]:
# stack so each image is one column in the data matrix
all_pics_c = [cropped_pics[i][j].flatten() for i in range(len(cropped_pics))
              for j in range(len(cropped_pics[i]))]
A_yf = np.transpose(np.asarray(all_pics_c))

print(A_yf.shape)


(32256, 2432)


In [17]:
# repeat for the averaged matrix
avg_pics_c = [cropped_avgs[i][j].flatten() for i in range(len(cropped_avgs))
              for j in range(len(cropped_avgs[i]))]
A_avg = np.transpose(np.asarray(avg_pics_c))

print(A_avg.shape)


(168, 7296)


In [18]:
%%time

C_avg = np.corrcoef(A_avg)


CPU times: user 20.7 ms, sys: 7.84 ms, total: 28.5 ms
Wall time: 20.9 ms


In [22]:
C_avg

array([[1.        , 0.99423979, 0.97958643, ..., 0.76016955, 0.75067853,
        0.73910668],
       [0.99423979, 1.        , 0.99431999, ..., 0.76026178, 0.74915701,
        0.73616058],
       [0.97958643, 0.99431999, 1.        , ..., 0.75820589, 0.74541027,
        0.73083476],
       ...,
       [0.76016955, 0.76026178, 0.75820589, ..., 1.        , 0.99349548,
        0.9788784 ],
       [0.75067853, 0.74915701, 0.74541027, ..., 0.99349548, 1.        ,
        0.99452001],
       [0.73910668, 0.73616058, 0.73083476, ..., 0.9788784 , 0.99452001,
        1.        ]])

In [8]:
one_person = [cropped_pics[i][0].flatten() for i in range(len(cropped_pics))]
A_1pers = np.transpose(np.asarray(one_person))


In [9]:
A_1pers.shape

(32256, 38)

In [10]:
ten_peeps = [cropped_pics[i][j].flatten() for i in range(len(cropped_pics))
             for j in range(10)]
A_10p = np.transpose(np.asarray(ten_peeps))


In [ ]:
power

## part (a)
**Power iterate** on the matrix of images to find the dominant eigenvector and eigenvalue. Compare to the leading order SVD mode.

In [34]:
# on the average matrix
l_avg, v_avg, n_avg = power_iteration(C_avg)


In [35]:
# perform (economy) SVD
[Ut, St, VTt] = np.linalg.svd(C_avg, full_matrices=False)


In [37]:
print(St[0], l_avg)

100.09425814718432 100.09425814718422


#### Plots???

In [ ]:
# TODO? But I don't actually think I need to


## part (b)
Use **randomized sampling** to reproduce the SVD matrices: U, $\Sigma$, and V

## part (c)
Copare the randomized modes to the true modes along with the singular value decay as a function of the number of randomized samples